In [ ]:
%matplotlib inline
from ipyparallel import Client, error
cluster=Client(profile="mpi")
view=cluster[:]
view.block=True

try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# DOEDriver

DOEDriver facilitates performing a design of experiments (DOE) with your OpenMDAO model. It will run your model multiple times with different values for the design variables depending on the selected input generator. A number of generators are available, each with its own parameters that can be specified when it is instantiated:

  - UniformGenerator
  - FullFactorialGenerator
  - PlackettBurmanGenerator
  - BoxBehnkenGenerator
  - LatinHypercubeGenerator
  - CSVGenerator
  - ListGenerator

See the [source documentation](../../../_srcdocs/packages/drivers/doe_generators) of these generators for details.

```{Note}
`FullFactorialGenerator`, `PlackettBurmanGenerator`, `BoxBehnkenGenerator` and `LatinHypercubeGenerator` are provided via the [pyDOE3](https://pypi.org/project/pyDOE3/) package, which is an updated version of [pyDOE](https://pythonhosted.org/pyDOE/). See the original [pyDOE](https://pythonhosted.org/pyDOE/) page for information on those algorithms.
```

The generator instance may be supplied as an argument to the `DOEDriver` or as an option.

## DOEDriver Options

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.drivers.doe_driver.DOEDriver")

## DOEDriver Constructor

The call signature for the `DOEDriver` constructor is:

```{eval-rst}
    .. automethod:: openmdao.drivers.doe_driver.DOEDriver.__init__
       :noindex:
```  

## Simple Example

`UniformGenerator` implements the simplest method and will generate a requested number of samples randomly selected from a uniform distribution across the valid range for each design variable. This example demonstrates its use with a model built on the [Paraboloid](../../../basic_user_guide/single_disciplinary_optimization/first_analysis) Component. An [SqliteRecorder](../../../_srcdocs/packages/recorders/sqlite_recorder) is used to capture the cases that were generated. We can see that that the model was evaluated at random values of x and y between -10 and 10, per the lower and upper bounds of those design variables.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src16", get_code("openmdao.test_suite.components.paraboloid.Paraboloid"), display=False)

:::{Admonition} `Paraboloid` class definition 
:class: dropdown

{glue:}`code_src16`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])

model.add_design_var('x', lower=-10, upper=10)
model.add_design_var('y', lower=-10, upper=10)
model.add_objective('f_xy')

prob.driver = om.DOEDriver(om.UniformGenerator(num_samples=5))
prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob.setup()

prob.set_val('x', 0.0)
prob.set_val('y', 0.0)

prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")
cases = cr.list_cases('driver')

In [ ]:
values = []
for case in cases:
    outputs = cr.get_case(case).outputs
    values.append((outputs['x'], outputs['y'], outputs['f_xy']))

print("\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]))

In [ ]:
assert(len(cases) == 5)

import os
os.remove('cases.sql')

## Running a DOE in Parallel

In a parallel processing environment, it is possible for `DOEDriver` to run cases concurrently. This is done by setting the `run_parallel` option to True as shown in the following example and running your script using MPI.

Here we are using the `FullFactorialGenerator` with 3 levels to generate inputs for our Paraboloid model. With two inputs, $3^2=9$ cases have been generated. In this case we are running on four processors and have specified `options['run_parallel']=True` to run cases on all available processors. The cases have therefore been split with 3 cases run on the first processor and 2 cases on each of the other processors.

Note that, when running in parallel, the `SqliteRecorder` will generate a separate case file for each processor on which cases are recorded. The case files will have a suffix indicating the recording rank and a message will be displayed indicating the file name, as seen in the example.

```{note}
This feature requires MPI, and may not be able to be run on Colab or Binder.
```

In [ ]:
%%px

import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()

prob.model.add_subsystem('comp', Paraboloid(), promotes=['x', 'y', 'f_xy'])
prob.model.add_design_var('x', lower=0.0, upper=1.0)
prob.model.add_design_var('y', lower=0.0, upper=1.0)
prob.model.add_objective('f_xy')

prob.driver = om.DOEDriver(om.FullFactorialGenerator(levels=3))
prob.driver.options['run_parallel'] = True
prob.driver.options['procs_per_model'] = 1

prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob.setup()
prob.run_driver()
prob.cleanup()

In [ ]:
%%px

# check recorded cases from each case file
from mpi4py import MPI
rank = MPI.COMM_WORLD.rank

filename = "cases.sql_%d" % rank

cr = om.CaseReader(filename)
cases = cr.list_cases('driver', out_stream=None)
print(cases)

In [ ]:
%%px

values = []
for case in cases:
    outputs = cr.get_case(case).outputs
    values.append((outputs['x'], outputs['y'], outputs['f_xy']))

print("\n"+"\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]))

In [ ]:
%%px

import numpy as np

assert(len(cases) == 3 if rank == 0 else 2)
if rank == 0:
    expected = [
        {'x': np.array([0.]), 'y': np.array([0.]), 'f_xy': np.array([22.00])},
        {'x': np.array([.5]), 'y': np.array([.5]), 'f_xy': np.array([23.75])},
        {'x': np.array([1.]), 'y': np.array([1.]), 'f_xy': np.array([27.00])}]
elif rank == 1:
    expected = [
        {'x': np.array([.5]), 'y': np.array([0.]), 'f_xy': np.array([19.25])},
        {'x': np.array([1.]), 'y': np.array([.5]), 'f_xy': np.array([21.75])}]
elif rank == 2:
    expected = [
        {'x': np.array([1.]), 'y': np.array([0.]), 'f_xy': np.array([17.00])},
        {'x': np.array([0.]), 'y': np.array([1.]), 'f_xy': np.array([31.00])}]
else:
    expected = [
        {'x': np.array([0.]), 'y': np.array([.5]), 'f_xy': np.array([26.25])},
        {'x': np.array([.5]), 'y': np.array([1.]), 'f_xy': np.array([28.75])}]


exp_values = []
for case in expected:
    exp_values.append((case['x'], case['y'], case['f_xy']))
expect_text = "\n"+"\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % vals_i for vals_i in exp_values])

assert("\n"+"\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]) == expect_text)
import os
if rank == 0:
    os.remove('cases.sql_0')
    os.remove('cases.sql_1')
    os.remove('cases.sql_2')
    os.remove('cases.sql_3')
    os.remove('cases.sql_meta')

## Running a DOE in Parallel with a Parallel Model

If the model that is being subjected to the DOE is also parallel, then the total number of processors should reflect the model size as well as the desired concurrency.

To illustrate this, we will demonstrate performing a DOE on a model based on the [ParallelGroup](../../core_features/working_with_groups/parallel_group.ipynb) example:

In [ ]:
%%px

class FanInGrouped(om.Group):
    """
    Topology where two components in a Group feed a single component
    outside of that Group.
    """

    def __init__(self):
        super().__init__()

        self.set_input_defaults('x1', 1.0)
        self.set_input_defaults('x2', 1.0)

        self.sub = self.add_subsystem('sub', om.ParallelGroup(),
                                      promotes_inputs=['x1', 'x2'])

        self.sub.add_subsystem('c1', om.ExecComp(['y=-2.0*x']),
                               promotes_inputs=[('x', 'x1')])
        self.sub.add_subsystem('c2', om.ExecComp(['y=5.0*x']),
                               promotes_inputs=[('x', 'x2')])

        self.add_subsystem('c3', om.ExecComp(['y=3.0*x1+7.0*x2']))

        self.connect("sub.c1.y", "c3.x1")
        self.connect("sub.c2.y", "c3.x2")

In this case, the model itself requires two processors, so in order to run cases concurrently we need to allocate at least four processors in total. We can allocate as many processors as we have available, however the number of processors must be a multiple of the number of processors per model, which is 2 here. Regardless of how many processors we allocate, we need to tell the `DOEDriver` that the model needs 2 processors, which is done by specifying `options['procs_per_model']=2`. From this, the driver figures out how many models it can run in parallel, which in this case is also 2.

The `SqliteRecorder` will record cases on the first two processors, which serve as the “root” processors for the parallel cases.

In [ ]:
%%px

import openmdao.api as om

prob = om.Problem(FanInGrouped())

prob.model.add_design_var('x1', lower=0.0, upper=1.0)
prob.model.add_design_var('x2', lower=0.0, upper=1.0)
prob.model.add_objective('c3.y')

prob.driver = om.DOEDriver(om.FullFactorialGenerator(levels=3))
prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

# the FanInGrouped model uses 2 processes, so we can run
# two instances of the model at a time, each using 2 of our 4 procs
prob.driver.options['run_parallel'] = True
prob.driver.options['procs_per_model'] = procs_per_model = 2

prob.setup()
prob.run_driver()
prob.cleanup()

# a separate case file will be written by rank 0 of each parallel model
# (the top two global ranks)
rank = prob.comm.rank

num_models = prob.comm.size // procs_per_model

if rank < num_models:
    filename = "cases.sql_%d" % rank

    cr = om.CaseReader(filename)
    cases = cr.list_cases('driver', out_stream=None)

    values = []
    for case in cases:
        outputs = cr.get_case(case).outputs
        values.append((outputs['x1'], outputs['x2'], outputs['c3.y']))

    print("\n"+"\n".join(["x1: %5.2f, x2: %5.2f, c3.y: %6.2f" % (x1, x2, y) for x1, x2, y in values]))

In [ ]:
%%px
if rank == 0:
    expected = [
            {'x1': np.array([0.]), 'x2': np.array([0.]), 'c3.y': np.array([0.00])},
            {'x1': np.array([1.]), 'x2': np.array([0.]), 'c3.y': np.array([-6.00])},
            {'x1': np.array([.5]), 'x2': np.array([.5]), 'c3.y': np.array([14.50])},
            {'x1': np.array([0.]), 'x2': np.array([1.]), 'c3.y': np.array([35.00])},
            {'x1': np.array([1.]), 'x2': np.array([1.]), 'c3.y': np.array([29.00])}]
elif rank == 1:
    expected = [
            {'x1': np.array([.5]), 'x2': np.array([0.]), 'c3.y': np.array([-3.00])},
            {'x1': np.array([0.]), 'x2': np.array([.5]), 'c3.y': np.array([17.50])},
            {'x1': np.array([1.]), 'x2': np.array([.5]), 'c3.y': np.array([11.50])},
            {'x1': np.array([.5]), 'x2': np.array([1.]), 'c3.y': np.array([32.00])}]

if rank < 2:
    exp_values = []
    for idx, case in enumerate(expected):
        exp_values.append((case['x1'], case['x2'], case['c3.y']))

    expect_text = "\n"+"\n".join([
        "x1: %5.2f, x2: %5.2f, c3.y: %6.2f" % vals_i for vals_i in exp_values])
    assert("\n"+"\n".join(["x1: %5.2f, x2: %5.2f, c3.y: %6.2f" % (x1, x2, y) for x1, x2, y in values]) == expect_text)

if rank == 0:
    os.remove('cases.sql_0')
    os.remove('cases.sql_1')
    os.remove('cases.sql_meta')

## Using Prepared Cases

If you have a previously generated set of cases that you want to run using `DOEDriver`, there are a couple of ways to do that. The first is to provide those inputs via an external file in the CSV (comma separated values) format. The file should be organized with one column per design variable, with the first row containing the names of the design variables. The following example demonstrates how to use such a file to run a DOE using the `CSVGenerator`:

In [ ]:
# This cell creates a file called saved_cases.csv.

import json
import numpy as np

expected_csv = '\n'.join([
    " x ,   y",
    "0.0,  0.0",
    "0.5,  0.0",
    "1.0,  0.0",
    "0.0,  0.5",
    "0.5,  0.5",
    "1.0,  0.5",
    "0.0,  1.0",
    "0.5,  1.0",
    "1.0,  1.0",
])

with open('saved_cases.csv', 'w') as f:
    f.write(expected_csv)

expected = [
    {'x': np.array([0.]), 'y': np.array([0.]), 'f_xy': np.array([22.00])},
    {'x': np.array([.5]), 'y': np.array([0.]), 'f_xy': np.array([19.25])},
    {'x': np.array([1.]), 'y': np.array([0.]), 'f_xy': np.array([17.00])},

    {'x': np.array([0.]), 'y': np.array([.5]), 'f_xy': np.array([26.25])},
    {'x': np.array([.5]), 'y': np.array([.5]), 'f_xy': np.array([23.75])},
    {'x': np.array([1.]), 'y': np.array([.5]), 'f_xy': np.array([21.75])},

    {'x': np.array([0.]), 'y': np.array([1.]), 'f_xy': np.array([31.00])},
    {'x': np.array([.5]), 'y': np.array([1.]), 'f_xy': np.array([28.75])},
    {'x': np.array([1.]), 'y': np.array([1.]), 'f_xy': np.array([27.00])},
]

values = []
cases = []

for case in expected:
    values.append((case['x'], case['y'], case['f_xy']))
    # converting ndarray to list enables JSON serialization
    cases.append((('x', list(case['x'])), ('y', list(case['y']))))

expected_text = "\n".join([
    "x: %5.2f, y: %5.2f, f_xy: %6.2f" % vals_i for vals_i in values
])

expected_json = json.dumps(cases).replace(']]],', ']]],\n')
with open('cases.json', 'w') as f:
    f.write(expected_json)

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['x', 'y', 'f_xy'])

model.add_design_var('x', lower=0.0, upper=1.0)
model.add_design_var('y', lower=0.0, upper=1.0)
model.add_objective('f_xy')

prob.setup()

prob.set_val('x', 0.0)
prob.set_val('y', 0.0)

# this file contains design variable inputs in CSV format
with open('saved_cases.csv', 'r') as f:
    print(f.read())

In [ ]:
# run problem with DOEDriver using the CSV file
prob.driver = om.DOEDriver(om.CSVGenerator('saved_cases.csv'))
prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")
cases = cr.list_cases('driver', out_stream=None)

values = []
for case in cases:
    outputs = cr.get_case(case).outputs
    values.append((outputs['x'], outputs['y'], outputs['f_xy']))

print("\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]))

In [ ]:
assert("\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]) == expected_text)
os.remove('cases.sql')

The second method is to provide the data directly as a list of cases, where each case is a collection of name/value pairs for the design variables. You might use this method if you want to generate the cases programmatically via another algorithm or if the data is available in some format other than a CSV file and you can reformat it into this simple list structure. The `DOEGenerator` you would use in this case is the `ListGenerator`, but if you pass a list directly to the `DOEDriver` it will construct the `ListGenerator` for you. In the following example, a set of cases has been pre-generated and saved in JSON (JavaScript Object Notation) format. The data is decoded and provided to the `DOEDriver` as a list:

In [ ]:
# load design variable inputs from JSON file and decode into list
with open('cases.json', 'r') as f:
    json_data = f.read()

print(json_data)

In [ ]:
# create DOEDriver using provided list of cases
case_list = json.loads(json_data)
prob.driver = om.DOEDriver(case_list)

prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob.run_driver()
prob.cleanup()

# check the recorded cases
cr = om.CaseReader("cases.sql")
cases = cr.list_cases('driver', out_stream=None)

values = []
for case in cases:
    outputs = cr.get_case(case).outputs
    values.append((outputs['x'], outputs['y'], outputs['f_xy']))

print("\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]))

In [ ]:
assert(type(prob.driver.options['generator']) == om.ListGenerator)
assert("\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]) == expected_text)
import os
os.remove('cases.sql')
os.remove('cases.json')

```{Note}
When using pre-generated cases via `CSVGenerator` or `ListGenerator`, there is no enforcement of the declared bounds on a design variable as with the algorithmic generators.
```